In [106]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

# Load data

In [107]:
df = pd.read_csv('data/ecommerce_customer_data.csv')
df

,CustomerID,RegistrationDate,Age,Gender,IncomeLevel,Country,City,TotalPurchases,AverageOrderValue,CustomerLifetimeValue,...,SocialMediaEngagementRate,MobileAppUsage,CustomerServiceInteractions,AverageSatisfactionScore,EmailConversionRate,SocialMediaConversionRate,SearchEngineConversionRate,RepeatCustomer,PremiumMember,HasReturnedItems
0,NaN,2020-05-10,25.0,NaN,NaN,Other,Tokyo,4.0,15.886509,NaN,...,0.379694,High,0.0,6.240881,0.112955,0.239948,0.156776,Yes,Yes,No
1,CUST00002,2021-07-18,NaN,Prefer not to say,High,Germany,London,6.0,27.638853,181.725056,...,0.140988,Low,0.0,7.721917,0.300979,0.230821,0.290735,Yes,No,No
2,CUST00003,2021-02-04,43.0,NaN,High,France,NaN,10.0,161.739425,1810.555150,...,0.323660,Low,0.0,7.899042,NaN,0.132239,0.050505,Yes,No,Yes
3,CUST00004,2020-12-31,49.0,Female,High,Australia,Los Angeles,5.0,14.194263,86.219740,...,0.268428,Never,1.0,7.829459,0.074097,0.206644,0.281067,Yes,Yes,No
4,CUST00005,2022-06-27,29.0,Female,Very High,Australia,Tokyo,7.0,31147.427206,2112.575945,...,0.160427,High,NaN,7.777353,0.163438,0.232410,0.326645,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,CUST09996,2018-09-29,39.0,Female,High,Japan,New York,4.0,60.986389,212.992614,...,0.128575,High,1.0,6.094181,0.240422,0.293145,NaN,Yes,Yes,Yes
9996,CUST09997,2020-10-25,58.0,Prefer not to say,Low,France,Sydney,6.0,75.208556,549.352094,...,0.298521,Never,NaN,8.610345,0.056814,0.128591,0.070305,Yes,No,Yes
9997,CUST09998,2023-04-07,14.0,Other,Very High,USA,New York,1.0,37.196899,117.203509,...,0.126018,NaN,0.0,9.864245,0.355694,0.101445,0.161469,No,No,Yes
9998,CUST09999,2021-05-27,29.0,Other,High,Canada,Berlin,3.0,13.191952,-6.655247,...,0.600112,NaN,0.0,6.163032,0.222856,0.160716,0.431023,Yes,No,No


# Cleaning

## Simple replacements

In [108]:
def column_replace(df, column_name, from_value, to_value):
    # replace all instances of from_value with to_value for a given column
    df.loc[df[column_name] == from_value, column_name] = to_value

### Gender

Replace shortened text anomalies

In [109]:
column_replace(df, 'Gender', 'F', 'Female')
column_replace(df, 'Gender', 'M', 'Male')

Replace N/As

In [110]:
df['Gender'] = df['Gender'].fillna('Prefer not to say')

Check

In [111]:
int(df[df['Gender'] == 'F'].shape[0] + df[df['Gender'] == 'M'].shape[0] + df['Gender'].isna().sum())

0

## PCA replacements

Extracts dimensionally-reduced features with PCA, and then uses a regressor for prediction.

Based on https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html

In [116]:
def impute(df, column_name, from_columns):
    mask_unknown = df[column_name].isna()
    mask_known = ~mask_unknown

    X = df[from_columns]
    y = df[column_name]    
    X_known = X[mask_known]
    y_known = y[mask_known]

    numeric_pipeline = Pipeline([
        # NOTE, this is for handling N/A in INPUT FEATURES, not for our final imputation 
        ('imputer', SimpleImputer(strategy = 'median')), 
        ('scaler', StandardScaler()),
        # reduce to n - 1 components, this might require tweaking, but is a good start
        ('pca', PCA(n_components = len(from_columns) - 1))
    ])

    # regressor for final imputation
    model = RandomForestRegressor(
        n_estimators = 300,
        random_state = 42,
        n_jobs = -1
    )

    # data flow pipeline
    pipe = Pipeline([
        ('preprocess', ColumnTransformer([
            ('num', numeric_pipeline, from_columns)
        ])),
        ('regressor', model)
    ])

    # train the model
    pipe.fit(X_known, y_known)
    
    if mask_unknown.any():
        X_missing = X[mask_unknown]
        # inference: do the imputing
        y_pred = pipe.predict(X_missing)
        # write back imputed data to dataframe
        df.loc[mask_unknown, column_name] = y_pred

    return df

In [113]:
df['AverageSatisfactionScore'].isna().sum()

np.int64(496)

In [114]:
impute(df, 'AverageSatisfactionScore', ['TotalPurchases', 'AverageOrderValue', 'CustomerServiceInteractions'])

,CustomerID,RegistrationDate,Age,Gender,IncomeLevel,Country,City,TotalPurchases,AverageOrderValue,CustomerLifetimeValue,...,SocialMediaEngagementRate,MobileAppUsage,CustomerServiceInteractions,AverageSatisfactionScore,EmailConversionRate,SocialMediaConversionRate,SearchEngineConversionRate,RepeatCustomer,PremiumMember,HasReturnedItems
0,NaN,2020-05-10,25.0,Prefer not to say,NaN,Other,Tokyo,4.0,15.886509,NaN,...,0.379694,High,0.0,6.240881,0.112955,0.239948,0.156776,Yes,Yes,No
1,CUST00002,2021-07-18,NaN,Prefer not to say,High,Germany,London,6.0,27.638853,181.725056,...,0.140988,Low,0.0,7.721917,0.300979,0.230821,0.290735,Yes,No,No
2,CUST00003,2021-02-04,43.0,Prefer not to say,High,France,NaN,10.0,161.739425,1810.555150,...,0.323660,Low,0.0,7.899042,NaN,0.132239,0.050505,Yes,No,Yes
3,CUST00004,2020-12-31,49.0,Female,High,Australia,Los Angeles,5.0,14.194263,86.219740,...,0.268428,Never,1.0,7.829459,0.074097,0.206644,0.281067,Yes,Yes,No
4,CUST00005,2022-06-27,29.0,Female,Very High,Australia,Tokyo,7.0,31147.427206,2112.575945,...,0.160427,High,NaN,7.777353,0.163438,0.232410,0.326645,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,CUST09996,2018-09-29,39.0,Female,High,Japan,New York,4.0,60.986389,212.992614,...,0.128575,High,1.0,6.094181,0.240422,0.293145,NaN,Yes,Yes,Yes
9996,CUST09997,2020-10-25,58.0,Prefer not to say,Low,France,Sydney,6.0,75.208556,549.352094,...,0.298521,Never,NaN,8.610345,0.056814,0.128591,0.070305,Yes,No,Yes
9997,CUST09998,2023-04-07,14.0,Other,Very High,USA,New York,1.0,37.196899,117.203509,...,0.126018,NaN,0.0,9.864245,0.355694,0.101445,0.161469,No,No,Yes
9998,CUST09999,2021-05-27,29.0,Other,High,Canada,Berlin,3.0,13.191952,-6.655247,...,0.600112,NaN,0.0,6.163032,0.222856,0.160716,0.431023,Yes,No,No


In [115]:
df['AverageSatisfactionScore'].isna().sum()

np.int64(0)